In [2]:
from __future__ import print_function


from tqdm import tqdm
import os
import sys
import time
import random
import pprint
import datetime
import dateutil.tz
import argparse
import numpy as np
import pandas as pd
from misc.config import Config
cfg = Config()
import pickle
import torch
import torchvision.transforms as transforms
pd.options.display.max_colwidth=0

In [3]:
proj_ds=pd.read_csv('../data/indiana_projections.csv')
repo_ds=pd.read_csv('../data/indiana_reports.csv')

# display(proj_ds.head(5),proj_ds.shape)
# display(repo_ds.sort_values(by='uid').head(5),repo_ds.shape)

merged_ds = pd.merge(left=repo_ds,right=proj_ds, how='inner', on='uid')
# display(merged_ds.head())
frontal_ds = merged_ds[merged_ds['projection']=='Frontal']
frontal_ds

,uid,MeSH,Problems,image,indication,comparison,findings,impression,filename,projection
0,1,normal,normal,Xray Chest PA and Lateral,Positive TB test,None.,The cardiac silhouette and mediastinum size are within normal limits. There is no pulmonary edema. There is no focal consolidation. There are no XXXX of a pleural effusion. There is no evidence of pneumothorax.,Normal chest x-XXXX.,1_IM-0001-4001.dcm.png,Frontal
2,2,Cardiomegaly/borderline;Pulmonary Artery/enlarged,Cardiomegaly;Pulmonary Artery,"Chest, 2 views, frontal and lateral",Preop bariatric surgery.,None.,Borderline cardiomegaly. Midline sternotomy XXXX. Enlarged pulmonary arteries. Clear lungs. Inferior XXXX XXXX XXXX.,No acute pulmonary findings.,2_IM-0652-1001.dcm.png,Frontal
4,3,normal,normal,Xray Chest PA and Lateral,"rib pain after a XXXX, XXXX XXXX steps this XXXX. Pain to R back, R elbow and R rib XXXX, no previous heart or lung hx, non-XXXX, no hx ca",NaN,NaN,"No displaced rib fractures, pneumothorax, or pleural effusion identified. Well-expanded and clear lungs. Mediastinal contour within normal limits. No acute cardiopulmonary abnormality identified.",3_IM-1384-1001.dcm.png,Frontal
6,4,"Pulmonary Disease, Chronic Obstructive;Bullous Emphysema;Pulmonary Fibrosis/interstitial;Cicatrix/lung/upper lobe/left;Opacity/lung/apex/left/irregular;Opacity/lung/upper lobe/right/streaky;Opacity/pulmonary alveoli;Opacity/lung/bilateral/interstitial/diffuse","Pulmonary Disease, Chronic Obstructive;Bullous Emphysema;Pulmonary Fibrosis;Cicatrix;Opacity;Opacity;Opacity;Opacity","PA and lateral views of the chest XXXX, XXXX at XXXX hours",XXXX-year-old XXXX with XXXX.,None available,"There are diffuse bilateral interstitial and alveolar opacities consistent with chronic obstructive lung disease and bullous emphysema. There are irregular opacities in the left lung apex, that could represent a cavitary lesion in the left lung apex.There are streaky opacities in the right upper lobe, XXXX scarring. The cardiomediastinal silhouette is normal in size and contour. There is no pneumothorax or large pleural effusion.","1. Bullous emphysema and interstitial fibrosis. 2. Probably scarring in the left apex, although difficult to exclude a cavitary lesion. 3. Opacities in the bilateral upper lobes could represent scarring, however the absence of comparison exam, recommend short interval followup radiograph or CT thorax to document resolution.",4_IM-2050-1001.dcm.png,Frontal
8,5,Osteophyte/thoracic vertebrae/multiple/small;Thickening/pleura/apex/bilateral;Lung/hyperdistention/mild,Osteophyte;Thickening;Lung,Xray Chest PA and Lateral,Chest and nasal congestion.,NaN,"The cardiomediastinal silhouette and pulmonary vasculature are within normal limits. There is no pneumothorax or pleural effusion. There are no focal areas of consolidation. Cholecystectomy clips are present. Small T-spine osteophytes. There is biapical pleural thickening, unchanged from prior. Mildly hyperexpanded lungs.",No acute cardiopulmonary abnormality.,5_IM-2117-1003002.dcm.png,Frontal
...,...,...,...,...,...,...,...,...,...,...
7456,3995,Lung/hyperdistention/mild;Diaphragm/bilateral/flattened;Cicatrix/lung/middle lobe/right/mild;Pulmonary Atelectasis/middle lobe/right/mild;Pulmonary Emphysema,Lung;Diaphragm;Cicatrix;Pulmonary Atelectasis;Pulmonary Emphysema,Xray Chest PA and Lateral,"Nausea, vomiting x2 weeks. Dialysis patient.","XXXX, XXXX.","The cardiomediastinal silhouette and pulmonary vasculature are within normal limits in size. The lungs are mildly hyperinflated with flattening of the diaphragms and expansion of the retrosternal clear space. Compared with prior exam, there has been interval resolution of previously demonstrated bibasilar infiltrates. There is minimal XXXX scarring or atelectasis in the right midlung. There is no XXXX focal airspace disease. There is no pneumothorax or pleural effusion. There are no acute bony findings.",1. Interval resolution of bibasilar airspace disease. 2. Hyperinflation with c

In [1]:
import pickle
with open('../data/cleaned_dataset.pickle','rb') as f:
    dataset = pickle.load(f)

In [5]:
dataset.keys()

dict_keys(['data_dict', 'data_split', 'word2idx', 'idx2word', 'classes'])

In [3]:
train_uids = dataset['data_split']['train_uids']
val_test_uids = dataset['data_split']['val_test_uids']
len(train_uids),len(val_test_uids)

(3166, 500)

In [ ]:
train_uids
val_test_uids

In [7]:
images=[]
for uid in dataset['data_dict'].keys():
    images+=dataset['data_dict'][uid]['filenames']


In [8]:
len(images)

3794

In [8]:
# train_uids_cls = list(set(dataset['classes'].keys()).intersection(set(train_uids)))
# np.array(train_uids_cls)

In [9]:
# val_uids_cls = list(set(dataset['classes'].keys()).intersection(set(val_uids)))

In [10]:
# test_uids_cls = list(set(dataset['classes'].keys()).intersection(set(test_uids)))

In [11]:
# val_test_uids = np.array(list(val_uids)+list(test_uids))
# dataset['data_split']['val_test_uids'] = val_test_uids

In [13]:
iu_classes_dict = {'no finding':7
                             ,'atelectasis':0
                             ,'cardiomegaly':1
                             ,'effusion':2
                             ,'emphysema':3
                             ,'infiltrate':4
                             ,'nodule':5
                             ,'thickening':6}
iu_classes = list(iu_classes_dict.keys())
iu_classes

['no finding',
 'atelectasis',
 'cardiomegaly',
 'effusion',
 'emphysema',
 'infiltrate',
 'nodule',
 'thickening']

In [51]:

uid_prob_mapping = {}
# uid_mesh_mapping = {}
for uid in tqdm(val_test_uids):
#     print(uid)
    problems = frontal_ds[frontal_ds.uid==uid]['Problems'].values[0].lower().strip().split(';')
    p_probs = []
    for cls in iu_classes:
        for p in problems:
            if cls in p:
                p_probs+=[cls]
    if (len(p_probs)==0):
        p_probs = ['no finding']
#     p_probs = [x for x in problems if x in iu_classes]
#     meshs = repo_ds[repo_ds.uid==key]['MeSH'].values[0].lower().split(';')
    uid_prob_mapping[uid]=p_probs
#     uid_mesh_mapping[key]=meshs

100%|██████████| 500/500 [00:00<00:00, 1748.11it/s]


In [52]:
# uid_prob_mapping

In [53]:
iu_problems = []
# iu_meshs = []
for key in tqdm(uid_prob_mapping.keys()):
    iu_problems+=uid_prob_mapping[key]
#     iu_meshs+=uid_mesh_mapping[key]

100%|██████████| 500/500 [00:00<00:00, 925894.92it/s]


In [54]:
# iu_problems

In [55]:
iu_prob_count = {}
for i in range(len(iu_problems)):    
    if iu_problems[i] not in iu_prob_count.keys():
        iu_prob_count[iu_problems[i]] = 1
    else:
        iu_prob_count[iu_problems[i]] += 1

In [50]:
df_iu_prob_count = pd.DataFrame.from_dict(iu_prob_count,orient='index',columns=['count']).reset_index().sort_values(by=['count'], ascending=True).reset_index()[['index','count']]
'test',df_iu_prob_count,'total:',df_iu_prob_count['count'].sum()

('test',
           index  count
 0  thickening    46   
 1  infiltrate    53   
 2  emphysema     86   
 3  nodule        103  
 4  effusion      137  
 5  atelectasis   265  
 6  cardiomegaly  290  
 7  no finding    2422 ,
 'total:',
 3402)

In [56]:
df_iu_prob_count = pd.DataFrame.from_dict(iu_prob_count,orient='index',columns=['count']).reset_index().sort_values(by=['index','count'], ascending=True)
'val',df_iu_prob_count,df_iu_prob_count.shape

('val',
           index  count
 1  atelectasis   45   
 4  cardiomegaly  29   
 3  effusion      17   
 5  emphysema     18   
 6  infiltrate    8    
 0  no finding    406  
 2  nodule        9    
 7  thickening    7    ,
 (8, 2))

In [1]:
df_iu_prob_count = pd.DataFrame.from_dict(iu_prob_count,orient='index',columns=['count']).reset_index().sort_values(by=['index','count'], ascending=True)
'train',df_iu_prob_count,df_iu_prob_count.shape

NameError: name 'pd' is not defined

In [334]:
iu_classes = [
    'atelectasis'
    ,'cardiomegaly'
    ,'effusion'
    ,'infiltrate'
    ,'nodule'
    ,'emphysema'
    ,'thickening'
]
len(iu_classes)

7

In [374]:
uid_prob_mapping = {}
# uid_mesh_mapping = {}
for uid in tqdm(dataset['data_dict'].keys()):
    problems = frontal_ds[frontal_ds.uid==uid]['Problems'].values[0].lower().strip().split(';')
    p_probs = []
    for cls in iu_classes:
        for p in problems:
            if cls in p:
                p_probs+=[cls]
    if (len(p_probs)==0):
        p_probs = ['no finding']
#     p_probs = [x for x in problems if x in iu_classes]
#     meshs = repo_ds[repo_ds.uid==key]['MeSH'].values[0].lower().split(';')
    uid_prob_mapping[uid]=[p_probs]
#     uid_mesh_mapping[key]=meshs

100%|██████████| 3666/3666 [00:01<00:00, 2069.58it/s]


In [375]:
df_iu_classes = pd.DataFrame.from_dict(uid_prob_mapping,orient='index',columns=['classes'])
df_iu_classes

,classes
1,[no finding]
2,[cardiomegaly]
3,[no finding]
4,[emphysema]
5,[thickening]
...,...
3995,"[atelectasis, emphysema]"
3996,[no finding]
3997,[no finding]
3998,[no finding]


In [376]:
dataset['classes'] = df_iu_classes.to_dict()['classes']
dataset['classes']

{1: ['no finding'],
 2: ['cardiomegaly'],
 3: ['no finding'],
 4: ['emphysema'],
 5: ['thickening'],
 6: ['no finding'],
 7: ['atelectasis'],
 8: ['no finding'],
 9: ['no finding'],
 10: ['no finding'],
 11: ['no finding'],
 12: ['no finding'],
 13: ['no finding'],
 14: ['no finding'],
 15: ['no finding'],
 17: ['no finding'],
 18: ['effusion'],
 19: ['no finding'],
 20: ['no finding'],
 21: ['no finding'],
 22: ['no finding'],
 23: ['no finding'],
 24: ['no finding'],
 25: ['effusion', 'effusion', 'emphysema'],
 26: ['no finding'],
 27: ['no finding'],
 28: ['atelectasis'],
 29: ['cardiomegaly'],
 30: ['no finding'],
 31: ['no finding'],
 32: ['no finding'],
 33: ['no finding'],
 34: ['no finding'],
 35: ['emphysema'],
 36: ['no finding'],
 37: ['no finding'],
 38: ['no finding'],
 39: ['no finding'],
 40: ['emphysema'],
 41: ['no finding'],
 42: ['no finding'],
 43: ['no finding'],
 44: ['no finding'],
 45: ['cardiomegaly'],
 46: ['no finding'],
 47: ['thickening'],
 48: ['nodule'],


In [338]:
# mimic_diseases = ['Atelectasis'
# ,'Cardiomegaly'
# ,'Consolidation'
# ,'Edema'
# ,'Enlarged Cardiomediastinum'
# ,'Fracture'
# ,'Lung Lesion'
# ,'Lung Opacity'
# ,'No Finding'
# ,'Pleural Effusion'
# ,'Pleural Other'
# ,'Pneumonia'
# ,'Pneumothorax'
# ,'Support Devices']

In [339]:
# len(mimic_diseases)

In [340]:
iu_problems = []
# iu_meshs = []
for key in tqdm(uid_prob_mapping.keys()):
    iu_problems+=uid_prob_mapping[key]
#     iu_meshs+=uid_mesh_mapping[key]

100%|██████████| 3666/3666 [00:00<00:00, 1394424.45it/s]


In [341]:
len(set(iu_problems))

8

In [343]:
df_iu_prob_count = pd.DataFrame.from_dict(iu_prob_count,orient='index',columns=['count']).reset_index().sort_values(by=['index','count'], ascending=True)

In [345]:
df_iu_prob_count

,index,count
4,atelectasis,310
1,cardiomegaly,319
5,effusion,154
2,emphysema,104
7,infiltrate,61
0,no finding,2828
6,nodule,112
3,thickening,53


In [119]:
pd.options.display.max_rows=2000